# Tensorflow MNIST Model

 * Wrap a Tensorflow MNIST python model for use as a prediction microservice in seldon-core
 
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install seldon-core
```

## Train locally
 

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
import tensorflow as tf

if __name__ == '__main__':
    
    x = tf.placeholder(tf.float32, [None,784], name="x")

    W = tf.Variable(tf.zeros([784,10]))
    b = tf.Variable(tf.zeros([10]))

    y = tf.nn.softmax(tf.matmul(x,W) + b, name="y")

    y_ = tf.placeholder(tf.float32, [None, 10])


    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

    init = tf.initialize_all_variables()

    sess = tf.Session()
    sess.run(init)

    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict = {x: mnist.test.images, y_:mnist.test.labels}))

    saver = tf.train.Saver()

    saver.save(sess, "model/deep_mnist_model")



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

Wrap model using s2i

In [2]:
!s2i build . seldonio/seldon-core-s2i-python37:0.13 deep-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl


Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Stored in directory: /root/.cache/pip/wheels/d7/de/2e/efa132238792efb6459a96e85916ef8597fcb3d2ae51590dfd
Stored in directory: /root/.cache/pip/wheels/a7/15/a0/0a0561549ad11cdc1bc8fa1191a353efd30facf6bfb507aefc
Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Stored in directory: /root/.cache/pip/wheels/2c/b1/94/43d03e130b929aae7ba3f8d15cbd7bc0d1cb5bb38a5c721833
Successfully built termcolor wrapt absl-py gast opt-einsum
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [3]:
!docker run --name "mnist_predictor" -d --rm -p 5000:5000 deep-mnist:0.1

67e96129d9f949157db068c202edb8e91de2b1a46444172311b329416bb0385d


Send some random features that conform to the contract

In [4]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[0.097 0.488 0.26  0.36  0.435 0.363 0.167 0.914 0.241 0.155 0.175 0.128
  0.539 0.681 0.821 0.34  0.09  0.914 0.569 0.177 0.836 0.059 0.706 0.354
  0.882 0.25  0.349 0.059 0.109 0.691 0.452 0.673 0.961 0.133 0.216 0.629
  0.275 0.159 0.758 0.74  0.545 0.124 0.891 0.72  0.162 0.244 0.435 0.351
  0.945 0.443 0.41  0.618 0.635 0.293 0.165 0.066 0.35  0.564 0.335 0.292
  0.744 0.517 0.663 0.784 0.35  0.322 0.501 0.31  0.146 0.238 0.482 0.878
  0.015 0.655 0.966 0.256 0.092 0.878 0.105 0.601 0.64  0.929 0.605 0.391
  0.249 0.158 0.004 0.973 0.743 0.054 0.05  0.65  0.061 0.338 0.342 0.505
  0.757 0.223 0.732 0.751 0.356 0.199 0.274 0.348 0.839 0.61  0.119 0.619
  0.401 0.969 0.147 0.123 0.663 0.658 0.37  0.842 0.988 0.1   0.272 0.126
  0.815 0.826 0.742 0.472 0.317 0.109 0.258 0.183 0.396 0.475 0.645 0.284
  0.973 0.967 0.519 0.938 0.499 0.45  0.198 0.987 0.75  0.515 0.703 0.754
  0.875 0.655 0.039 0.921 0.951 0.361 0.268 0.794

In [5]:
!docker rm mnist_predictor --force

mnist_predictor


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [6]:
!minikube start --memory 4096

😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


In [1]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=~kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [2]:
!helm init

$HELM_HOME has been configured at /home/joel/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation


In [3]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

deployment "tiller-deploy" successfully rolled out


In [4]:
!helm install ../../../helm-charts/seldon-core-operator --name seldon-core --set usageMetrics.enabled=true   --namespace seldon-system

NAME:   seldon-core
LAST DEPLOYED: Thu Nov 28 12:08:16 2019
NAMESPACE: seldon-system
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRole
NAME                     AGE
seldon-manager-css-role  1s
seldon-manager-role      1s
seldon-manager-sas-role  1s
seldon-proxy-role        1s

==> v1/ClusterRoleBinding
NAME                            AGE
seldon-manager-css-rolebinding  1s
seldon-manager-rolebinding      1s
seldon-manager-sas-rolebinding  1s
seldon-proxy-rolebinding        1s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-config            2     1s
seldon-spartakus-config  1     1s

==> v1/Deployment
NAME                        READY  UP-TO-DATE  AVAILABLE  AGE
seldon-controller-manager   0/1    1           0          1s
seldon-spartakus-volunteer  0/1    1           0          1s

==> v1/Pod(related)
NAME                                         READY  STATUS             RESTARTS  AGE
seldon-controller-manager-88bdbc885-dqkdm    0/1    ContainerCreating  0         1s
seldon-sp

In [5]:
!kubectl rollout status deploy/seldon-controller-manager -n seldon-system

Waiting for deployment "seldon-controller-manager" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-controller-manager" successfully rolled out


## Setup Ingress
Please note: There are reported gRPC issues with ambassador (see https://github.com/SeldonIO/seldon-core/issues/473).

In [6]:
!helm install stable/ambassador --name ambassador --set crds.keep=false

NAME:   ambassador
LAST DEPLOYED: Thu Nov 28 12:08:32 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Deployment
NAME        READY  UP-TO-DATE  AVAILABLE  AGE
ambassador  0/3    3           0          0s

==> v1/Pod(related)
NAME                         READY  STATUS             RESTARTS  AGE
ambassador-77f944f86f-dqk9l  0/1    ContainerCreating  0         0s
ambassador-77f944f86f-gjfls  0/1    ContainerCreating  0         0s
ambassador-77f944f86f-qlbtr  0/1    ContainerCreating  0         0s

==> v1/Service
NAME              TYPE          CLUSTER-IP      EXTERNAL-IP  PORT(S)                     AGE
ambassador        LoadBalancer  10.101.184.250  <pending>    80:30587/TCP,443:32046/TCP  0s
ambassador-admin  ClusterIP     10.104.170.192  <none>       8877/TCP                    0s

==> v1/ServiceAccount
NAME        SECRETS  AGE
ambassador  1        0s

==> v1beta1/ClusterRole
NAME             AGE
ambassador       0s
ambassador-crds  0s

==> v1beta1/ClusterRoleBinding
NAME   

In [7]:
!kubectl rollout status deployment.apps/ambassador

Waiting for deployment "ambassador" rollout to finish: 0 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 1 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 2 of 3 updated replicas are available...
deployment "ambassador" successfully rolled out


## Wrap Model and Test

In [14]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python37:0.13 deep-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
Build completed successfully


In [15]:
!kubectl create -f deep_mnist.json

seldondeployment.machinelearning.seldon.io/deep-mnist created


In [16]:
!kubectl rollout status deploy/deep-mnist-single-model-8969cc0

Waiting for deployment "deep-mnist-single-model-8969cc0" rollout to finish: 0 of 1 updated replicas are available...
deployment "deep-mnist-single-model-8969cc0" successfully rolled out


In [17]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    deep-mnist --namespace default -p

----------------------------------------
SENDING NEW REQUEST:

[[0.891 0.499 0.792 0.386 0.739 0.092 0.986 0.789 0.758 0.109 0.267 0.834
  0.542 0.025 0.02  0.16  0.6   0.324 0.71  0.515 0.346 0.943 0.109 0.455
  0.243 0.023 0.901 0.465 0.249 0.442 0.8   0.875 0.772 0.588 0.995 0.578
  0.254 0.461 0.885 0.068 0.157 0.489 0.486 0.951 0.123 0.052 0.72  0.515
  0.002 0.122 0.035 0.04  0.368 0.373 0.447 0.452 0.344 0.323 0.673 0.145
  0.206 0.721 0.749 0.759 0.184 0.86  0.061 0.304 0.429 0.621 0.723 0.916
  0.334 0.452 0.883 0.391 0.861 0.686 0.846 0.316 0.987 0.853 0.231 0.06
  0.763 0.215 0.119 0.001 0.234 0.717 0.765 0.42  0.71  0.605 0.166 0.192
  0.726 0.133 0.785 0.307 0.7   0.187 0.153 0.704 0.1   0.255 0.155 0.555
  0.89  0.312 0.884 0.85  0.361 0.992 0.061 0.781 0.678 0.537 0.292 0.007
  0.951 0.46  0.585 0.338 0.552 0.751 0.842 0.31  0.343 0.149 0.712 0.011
  0.209 0.412 0.859 0.859 0.159 0.366 0.644 0.196 0.347 0.527 0.823 0.737
  0.341 0.258 0.605 0.441 0.982 0.765 0.037 0.278 

In [16]:
!minikube delete

🔥  Deleting "minikube" from virtualbox ...
💔  The "minikube" cluster has been deleted.
